# 프롬프트 (`Prompt`)
`02_prompt.ipynb`

- LLM한테 주는 입력(지시, 맥락, 기억)
    - 지시: '~~해줘'
    - 맥락: Context - 현재 지시를 위해 제공하는 추가 정보
    - 기억: Memory - 지금까지 했던 대화 내용

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## `PromptTemplate`

- 단순히 1회성 명령을 내릴 때 사용

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model='gpt-4.1-nano')

In [ ]:
# 추후에 체인.invoke에서 {} 내부에 들어갈 말을 채워줘야 함.
template = '{country}의 수도가 어디인가요?'

# .from_template 메서드로 프롬프트 만들기
prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke 로 쓰게 됨)
prompt.format(country='대한민국')

'대한민국의 수도가 어디인가요?'

In [10]:
chain = prompt | llm

chain.invoke({'country': '대한민국'}).content

'대한민국의 수도는 서울입니다.'

### Partial Variable (부분 변수)

- 프롬프트에서 매개변수 기본값 사용하기

In [14]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template=template,
    input_variables=['c1'],
    partial_variables={
        'c2': '미국'
    }
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다') 

'한국과 미국의 수도는 각각 어디인가요?'

In [16]:
chain = prompt | llm
chain.invoke({'c1': 'kor', 'c2': 'us'}).content

'한국(대한민국, KOR)의 수도는 서울입니다.  \n미국(US)의 수도는 워싱턴 D.C.입니다.'

In [22]:
from datetime import datetime

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

prompt.format(n=3)

'오늘 날짜는 Sep 01입니다. 오늘 생일인 유명인 3명을 생년월일과 함께 나열해 주세요'

In [30]:
llm = ChatOpenAI(model='gpt-5-nano')

chain = prompt | llm
# 오늘 날짜 기준
print(chain.invoke({'n': 3}).content)
print('===')
print(chain.invoke({'n': 3, 'today': 'Jan 2'}).content)

오늘이 9월 1일이므로, 오늘 생일인 유명인 3명을 생년월일과 함께 정확히 알려드리려면 최신 정보를 확인해야 합니다. 인터넷 검색으로 확인해도 될까요? 허용해 주시면 Zendaya(1996-09-01) 외에 나머지 두 사람도 신뢰할 수 있는 출처로 확인해 바로 적어 드리겠습니다.
===
오늘 1월 2일 생일인 유명인 중 확인된 두 명은 다음과 같습니다.

- Isaac Asimov — 1920년 1월 2일
- Kate Bosworth — 1983년 1월 2일

세 번째 인물까지 포함해 3명을 확정해 드리려면 최신 자료 확인이 필요합니다. 원하시면 제가 바로 확인해서 세 번째 이름과 생년월일을 함께 알려드리겠습니다. 계속 진행할까요?


## `ChatPromptTemplate`
- 채팅을 주고받는 템플릿 생성용
- 대화 목록을 LLM에게 주입
- 하나의 Chat 은 `role` 과 `message` 로 구성됨

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

'Human: 한국의 수도느 어디?'

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '당신은 친절한 AI어시스트. 이름은 {name} 야.'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
    ]
)

# format vs format_messages
messages_str = chat_template.format(name='gaida', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name='gaida', user_input='이름이 뭐니?')
print(messages_str)
print(messages_cls)

System: 당신은 친절한 AI어시스트. 이름은 gaida이야.
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
[SystemMessage(content='당신은 친절한 AI어시스트. 이름은 gaida이야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [ ]:
# 한덩어리 텍스트
llm.invoke(messages_str).content
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

'제 이름은 gaida예요. 무엇을 도와드릴까요?'

In [42]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': '가이다', 'user_input': '내 이름은 유태영. 너와 나의 이름점을 봐줘'})

'안녕하세요, 유태영님! 저는 가이다이입니다. 이름점을 재미로 한 번 봐드릴게요. 이건 과학적 예언이 아니라, 놀이로 보는 간단한 해석입니다.\n\n간략한 이름점 결과\n- 합산 호감도: 약 82/100 (재미로 보는 수치입니다)\n- 전반적 분위기: 밝고 에너제틱한 인상과 차분한 이성적 접근이 잘 어우러지는 조합이라고 느껴집니다.\n\n유태영님의 이름 특징\n- 구성: 세 음절(유 / 태 / 영)으로 강한 리듬감이 있습니다.\n- 핵심 느낌: 밝은 에너지와 빛나는 이미지, 동시에 현실적이고 의지가 강한 면이 엿보입니다.\n\n가이다이의 이름 특징\n- 구성: 네 음절(가 / 이 / 다 / 이)로 명확하고 단정한 인상을 줍니다.\n- 핵심 느낌: 친근하고 도움을 주는 이미지, 문제를 차분하게 해결하려는 성향이 느껴집니다.\n\n두 이름의 궁합 포인트\n- 강점의 보완: 유태영님의 활력과 빛나는 기운이 가이다이의 차분함과 현실감과 만나 균형감을 만들어냅니다. 창의성과 실행력이 서로를 잘 보완합니다.\n- 의사소통의 흐름: 서로를 존중하고 배려하는 커뮤니케이션이 잘 맞는 편이지만, 때로는 직설적 표현이 오해를 살 수 있어 서로의 의도를 명확히 전하는 노력이 도움이 됩니다.\n\n조화롭게 맞추는 간단 팁\n- 서로의 강점을 살리려 애칭이나 부르는 이름을 편하게 사용해 보세요. 예: 가이, 유영 같은 간단한 호칭.\n- 의사소통 시 핵심 포인트를 먼저 간단히 정리한 후 대화하기.\n- 창의적 아이디어를 제시할 때는 먼저 공감과 칭찬으로 분위기를 살려 주세요.\n\n더 깊이 있게 탐구하고 싶으시다면 다른 방식으로도 이름점을 해볼 수 있어요. 예를 들어:\n- 자음·모음의 구성 분석으로 한층 세밀하게 보기\n- 글자 수와 음의 흐름에 따른 Compatibility 점수 조정\n- 한자 의미나 음양오행 관점으로 해석하기\n\n원하시는 방식이 있으면 알려주세요. 다른 이름들로도 playful하게 다시 해드릴게요.'

## 프롬프팅 방법
내가 원하는 답변을 구구절절 설명 X -> 예시를 주고 답변을 형태를 유도
- Zero-shot prompting : 예시 없이 질문만 -> 바로 답변
- One-shot prompting  : 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변
- **Few-shot prompting**  : 예시 여러개 + 질문 -> 예시들의 패턴을 일반화 해서 답변

In [60]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

chain = prompt | llm | StrOutputParser()

res = chain.invoke({'question': question})

print(res)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google이 창립된 연도는 언제인가요?
중간 답변: Google은 1998년에 설립되었습니다.
추가 질문: Bill Gates는 1998년에는 몇 살이었나요?
중간 답변: Bill Gates는 1998년에는 43세였습니다.
최종 답변은: 43세


## `langchain-hub`
다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [47]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [51]:
prompt = PromptTemplate.from_template(
    '내 첫 프롬프트 Push!'
)

hub.push('gaida1st/prompt-hub-test', prompt)

LangSmithUserError: Cannot create a prompt for another tenant.
Current tenant: None,
Requested tenant: gaida1st